In [12]:
import pandas as pd
from src.StreamPort.machine_learning.analyses import MachineLearningAnalyses

train_data = pd.read_csv("dev/train_features.csv")
train_metadata = pd.read_csv("dev/train_metadata.csv")
ana = MachineLearningAnalyses(variables = train_data, metadata = train_metadata)
print(ana)
ana.plot_data()


MachineLearningAnalyses 
  variables: 20 rows, 21 columns
  metadata: 20 rows, 14 columns



In [13]:
import numpy as np
from src.StreamPort.machine_learning.methods import MachineLearningMethodIsolationForestSklearn
from src.StreamPort.machine_learning.methods import MachineLearningScaleFeaturesScalerSklearn

scl = MachineLearningScaleFeaturesScalerSklearn(scaler_type="MaxAbsScaler")
iso = MachineLearningMethodIsolationForestSklearn(random_state = 30, contamination = "auto")

#scales and fits the model and upgrades analyses class to IsolationForestAnalyses
ana = scl.run(ana)
ana = iso.run(ana)
print(ana.__class__)

ana.plot_data()

<class 'src.StreamPort.machine_learning.analyses.IsolationForestAnalyses'>


In [14]:
fig_train_scores=ana.plot_scores()
fig_train_scores.write_image("dev/figures/fig_train_scores.png", width=1100, height= 350, scale = 3)
fig_train_scores.show()

In [15]:
test_data = pd.read_csv("dev/test_features.csv")
test_metadata = pd.read_csv("dev/test_metadata.csv")
ana.predict(test_data, test_metadata)
outliers_test = ana.test_prediction_outliers()
print(outliers_test)
fig_test_scores = ana.plot_scores()
fig_test_scores.write_image("dev/figures/fig_test_scores.png", width=1100, height= 350, scale = 3)
fig_test_scores.show()


    outlier     score
0      True -0.148199
1     False -0.026332
2     False -0.009801
3      True -0.066914
4     False -0.005658
5     False -0.008411
6     False  0.011134
7     False -0.006001
8      True -0.062248
9     False -0.018140
10    False -0.015529
11    False -0.034481
12    False -0.011909
13    False -0.027415
14    False -0.009811
15    False -0.015032
16    False -0.022056


In [16]:
fig_test_features = ana.plot_data()
fig_test_features.write_image("dev/figures/fig_test_features.png", width=1100, height= 350, scale = 3)
fig_test_features.show()

In [17]:
test_data = pd.read_csv("dev/test_2_features.csv")
test_metadata = pd.read_csv("dev/test_2_metadata.csv")
ana.predict(test_data, test_metadata)
outliers_test = ana.test_prediction_outliers()
print(outliers_test)
ana.plot_scores()

   outlier     score
0    False  0.022436
1     True -0.148199
2    False -0.026332
3    False -0.011732


In [18]:
test2_data = pd.read_csv("dev/test2_features.csv")
test2_metadata = pd.read_csv("dev/test2_metadata.csv")
ana.predict(test2_data, test2_metadata)
outliers_test2 = ana.test_prediction_outliers()
print(outliers_test2)
fig_test2_scores = ana.plot_scores()
fig_test2_scores.write_image("dev/figures/fig_test2_scores.png", width=1100, height= 350, scale = 3)
fig_test2_scores.show()

   outlier     score
0     True -0.076016
1     True -0.100667
2     True -0.142758
3     True -0.148199
4     True -0.144439


In [19]:
fig_test2_features = ana.plot_data()
fig_test2_features.write_image("dev/figures/fig_test2_features.png", width=1100, height= 350, scale = 3)
fig_test2_features.show()

In [20]:
#ana.add_prediction(add_outliers = False)
#ana.plot_scores()

In [21]:
# test_2_data = pd.read_csv("test_2_features.csv")
# test_2_metadata = pd.read_csv("test_2_metadata.csv")
# ana.predict(test_2_data, test_2_metadata)
# outliers_test_2 = ana.test_prediction_outliers()
# print(outliers_test_2)
# fig_test_scores=ana.plot_scores()
# fig_test_scores.write_image("dev/figures/fig_test_scores.png", width=1000, height= 350, scale = 3)
# fig_test_scores.show()

In [22]:
#ana.add_prediction(add_outliers = False)
#ana.plot_scores()